# Runfiles for NonLinLoc: Steps Vel2Grid + Grid2Time + NLLoc

This notebook was made to modify templates of runfiles for the NonLinLoc software. Their configuration is for a 3D velocity model, some command would have to be changed 2D.

1. First step is defining the travel-time grids. If it's for Vel2Grid: you can use the Matching_VelModel notebook to create 2 .txt files that will be needed to modify the template of the runfile. It's made to add the lines for 2 commands: VGGRID and LAYER. If it's for Vel2Grid3D: the line VGGRID is defined in SimulPS_Vel2Grid notebook and the VGINP command will be add instead of LAYER.
2. Second step, it modifies the command lines for the runfile made for location of events with NLLoc. All the parameters needed can be entered here and the modifications will be made on the template.
3. Final step, the templates are merged into 1 runfile and clear out of unecessary lines, and ready to be use!

In [154]:
import os
import glob
import json
base_dir = '/Users/lpapin/Documents/phd/projects/sw4/nonlinloc/'
base_dir_talapas = '/projects/amt/lpapin/SVI/NLLoc'

For now everything is in local but since NonLinLoc is on the cluster, let's try to keep a similar organization than the creator:

- NLLoc as the main folder
- NLLoc/model for the model grid
- NLLoc/time for the travel-time grid
- NLLoc/obs for the picks
- NLLoc/run for the runfiles
- NLLoc/loc for the locations

NB: #LP is added to each line modified thanks to this notebook to remember which lines were the original ones.

# 0. Generic statement

The TRANS command is defined as a generic statement so it's needed for any step of NonLinLoc, meaning it has to be in *any* runfile as well as CONTROL.

In [155]:
# Read the original .run file
with open(base_dir+'ttgrid_runfile_template.in', 'r') as file:
    lines = file.readlines()

# Get the parameters of the rectangle from the polygon
with open("svi_poly.json", 'r') as file:
    polygon = json.load(file)
latOrig = polygon[0]['olat']
longOrig = polygon[0]['olon']
firstStdParal = polygon[0]['plat1']
secondStdParal = polygon[0]['plat2']
rotAngle = 0.0

# Determine the command line for TRANS LAMBERT and replace
trans_command = 'TRANS LAMBERT WGS-84 ' + f'{latOrig} ' + f'{longOrig} ' + f'{firstStdParal} ' + f'{secondStdParal} ' + f'{rotAngle}'
for i, line in enumerate(lines):
    if line.startswith('TRANS LAMBERT '):
        lines[i] = trans_command + ' #LP' + '\n'
        break

with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('TRANS LAMBERT line has been modified successfully.')

TRANS LAMBERT line has been modified successfully.


# 1. Travel-time grids

Vel2Grid and Vel2Grid3D have slightly different input. They have common commands but the first one use LAYER while the second one has VGINP and VGCLIP. 

In [156]:
# Precise here which one is used to run the notebook entirely
which = 'vel2grid3d' # 'vel2grid
wave = 'P' # 'S'

## Vel2Grid

*Given a velocity model description, Vel2Grid generates a 3D model Grid header and buffer files containing velocities, slownesses or other model specification.*

First we're modifying the line giving the grid description VGGRID for Vel2Grid.

In [157]:
# Read the original .run file
with open(base_dir+'ttgrid_runfile_final.in', 'r') as file:
    lines = file.readlines()

# Read the second line directly from VGGRID_command.txt
with open(base_dir+'VGGRID_command.txt', 'r') as file:
    lines_to_add = file.readlines()
    vggrid_command = lines_to_add[1].strip()

# Find and replace the VGGRID line in the .run file
for i, line in enumerate(lines):
    if line.startswith('VGGRID'):
        lines[i] = vggrid_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('VGGRID line has been modified successfully.')

VGGRID line has been modified successfully.


Now we add the lines giving the velocity model layers. It is used for a type of model where the velocites don't change with lat and lon (hence the correction for the SVI project).

In [158]:
if which=='vel2grid':
    # Read the second line directly from LAYER_command.txt
    with open(base_dir+'LAYER_command.txt', 'r') as file:
        layer_lines = [line.strip() for line in file.readlines()[2:]]
    layer_lines = [f'{line} #LP' for line in layer_lines]
    
    # Find the line to insert the LAYER commands in the .run file
    for i, line in enumerate(lines):
        if line.startswith('#LAYER depth Vp_top Vp_grad Vs_top Vs_grad p_top p_grad'):
            insertion_point = i + 1  # Insert after this line
            break
    
    # Insert the LAYER lines at the specified point
    modified_lines = lines[:insertion_point] + layer_lines + lines[insertion_point:]
    
    # Write the modified content back to a new file (or overwrite the original)
    with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in modified_lines)
    
    print('LAYER lines have been added successfully.')
else:
    print('No LAYER lines added; Vel2Grid3D has been choosen')

No LAYER lines added; Vel2Grid3D has been choosen


## Vel2Grid3D

*Given an existing 3D velocity model defined by interpolation between control point nodes and created by velocity inversion programs such as SimulPS, Simul2000 and FDTomo to a 3D model grid. (outputs a 3D Grid) , Vel2Grid3D generates a 3D model Grid header and buffer files containing velocities, slownesses or other model specification.*

This is the part used for SVI, using SimulPS_Vel2Grid notebook to create the files.

In [159]:
# Get the filename of the SimulPS file depending on the wave
velocities_grids = glob.glob(base_dir + f'Savard*{wave}.txt')
filename_simulPS = [os.path.basename(grid) for grid in velocities_grids][0]

if which=='vel2grid3d':
    # Determine the filename for the SimulPS
    input_filename_simulPS = base_dir_talapas + '/model/' + filename_simulPS
    
    # Read the original .run file
    with open(base_dir+'ttgrid_runfile_final.in', 'r') as file:
        lines = file.readlines()
    
    vginp_command = 'VGINP ' + input_filename_simulPS + ' SIMUL2K ' + '0 0 0'
    # Find and replace the VGINP line in the .run file
    for i, line in enumerate(lines):
        if line.startswith('#VGINP'):
            lines[i] = vginp_command + ' #LP' + '\n'
            break
    
    # Write the modified content back to a new file (or overwrite the original)
    with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
        file.writelines(line.rstrip() + '\n' for line in lines)
    
    print('VGINP line has been modified successfully.')
else:
    print('No VGINP line added; Vel2Grid has been choosen')

VGINP line has been modified successfully.


## Grid2Time

*Given a velocity model grid, Grid2Time calculates the travel-time from a source point in a 3D grid to all other points in the grid. Optionally, the program also estimates the take-off angles for rays from each point in the grid to the source.*

The only line that needs to be modified is the one giving the coordinates of the stations that will be used for the locations. The INCLUDE line seems to be replacing the GTSOURCE lines.

In [160]:
# Determine the .txt file for station coordinates
input_filename_stas = base_dir_talapas + '/obs/stations_coords_svi.txt'

include_command = 'INCLUDE ' + input_filename_stas
# Find and replace the INCLUDE line in the .run file
for i, line in enumerate(lines):
    if line.startswith('INCLUDE'):
        lines[i] = include_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'ttgrid_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('INCLUDE line has been added successfully.')

INCLUDE line has been added successfully.


## Runfile ttgrid resulting

Just for clarity, the lines based on the type of Vel2Grid we are not using are removed. Then all lines for travel-time grid and locations are merge into 1 runfile that will be use as input file for all the software.

In [161]:
# Determine the filename 
input_file = 'ttgrid_runfile_final.in'
with open(input_file, 'r') as infile:
    lines = infile.readlines()

# Keep the text for the right parameter
modified_lines = []
inside_block = False
if which == 'vel2grid':
    for line in lines:
        if line.strip() == '## START -- Vel2Grid3D':
            inside_block = True  # Start removing lines
            continue
        if line.strip() == '## END -- Vel2Grid3D':
            inside_block = False  # Stop removing lines
            continue
        # Only keep the lines if we're not inside the block
        if not inside_block:
            modified_lines.append(line)
    with open(input_file, 'w') as outfile:
        outfile.writelines(modified_lines)
    print(f"Vel2Grid block removed from {input_file}.")
elif which == 'vel2grid3d':
    for line in lines:
        if line.strip() == '## START -- Vel2Grid':
            inside_block = True  # Start removing lines
            continue
        if line.strip() == '## END -- Vel2Grid':
            inside_block = False  # Stop removing lines
            continue
        # Only keep the lines if we're not inside the block
        if not inside_block:
            modified_lines.append(line)
    with open(input_file, 'w') as outfile:
        outfile.writelines(modified_lines)
    print(f"Vel2Grid3D block removed from {input_file}.")

Vel2Grid3D block removed from ttgrid_runfile_final.in.


In [162]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'ttgrid_runfile_final.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

CONTROL 1 54321
TRANS LAMBERT WGS-84 48.65 -123.75 48.0 49.3 0.0 #LP
VGOUT  ./model/layer
VGTYPE P
VGTYPE S
VGGRID 111 146 95 -55.0 -72.5 -1.0 1.0 1.0 1.0 SLOW_LEN #LP
VGINP /projects/amt/lpapin/SVI/NLLoc/model/Savard_lcc_P.txt SIMUL2K 0 0 0 #LP
GTFILES  ./model/layer  ./time/layer P
GTFILES  ./model/layer  ./time/layer S
GTMODE GRID3D ANGLES_NO
INCLUDE /projects/amt/lpapin/SVI/NLLoc/obs/stations_coords_svi.txt #LP
GT_PLFD  1.0e-3  0


**Now the input file to use Vel2Grid or Vel2Grid3D, and Grid2Timeis ready.**

# 2. Location of events

## NLLoc

*NLLoc performs earthquake locations in 3D models using non-linear search techniques.*

I need to define the commands LOCFILES, LOCGRID, and TRANS. Not changing for now the LOCSEARCH command but it may be needed.

In [163]:
# Determine the filenames for LOCFILES
picks_filename = 'LFE_cat_SVI.nlloc' # Bostock's families by Tim; then to be changed for mine + sorted for choosen stations
input_filename_obs = base_dir_talapas + '/obs/' + picks_filename # Picks 
input_filename_ttgrid = base_dir_talapas + '/time/layer' # Travel-time grids
output_filename_loc = base_dir_talapas + '/loc/SVI_Bostock/family' # Location of the events with Bostock's picks from Tim
#output_filename_loc = base_dir_talapas + '/loc/SVI' # Location of the events with my picks

# Read the original .run file
with open(base_dir+'nlloc_runfile_template.in', 'r') as file:
    lines = file.readlines()

locfiles_command = 'LOCFILES ' + input_filename_obs + ' NLLOC_OBS ' + input_filename_ttgrid + ' ' + output_filename_loc
# Find and replace the LOCFILES line in the .run file
for i, line in enumerate(lines):
    if line.startswith('LOCFILES'):
        lines[i] = locfiles_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'nlloc_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('LOCFILES line has been modified successfully.')

LOCFILES line has been modified successfully.


In [164]:
# Read the second line directly from LOCGRID_command.txt
with open(base_dir+'LOCGRID_command.txt', 'r') as file:
    lines_to_add = file.readlines()
    locgrid_command = lines_to_add[1].strip()

# Find and replace the LOCGRID line in the .run file
for i, line in enumerate(lines):
    if line.startswith('LOCGRID'):
        lines[i] = locgrid_command + ' #LP' + '\n'
        break

# Write the modified content back to a new file (or overwrite the original)
with open(base_dir+'nlloc_runfile_final.in', 'w') as file:
    file.writelines(line.rstrip() + '\n' for line in lines)

print('LOCGRID line has been modified successfully.')

LOCGRID line has been modified successfully.


## Runfile nlloc resulting

In [165]:
# What does it look like after modifying the .run file for the commands
with open(base_dir+'nlloc_runfile_final.in', 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    for line in non_comment_lines:
        print(line)

LOCSIG Lois Papin -- UO
LOCCOM Bostock_Loc
LOCFILES /projects/amt/lpapin/SVI/NLLoc/obs/LFE_cat_SVI.nlloc NLLOC_OBS /projects/amt/lpapin/SVI/NLLoc/time/layer /projects/amt/lpapin/SVI/NLLoc/loc/SVI_Bostock/family #LP
LOCHYPOUT SAVE_NLLOC_ALL SAVE_NLLOC_SUM SAVE_HYPOINV_SUM
LOCSEARCH  OCT 8 11 6 0.001 100000 10000 0 0 #Mine
LOCMETH EDT_OT_WT 9999.0 4 -1 -1 -1 -1 -1 1
LOCGAU 0.2 0.0
LOCGAU2 0.01 0.05 2.0
LOCPHASEID  P   P p Pn Pg
LOCPHASEID  S   S s Sn Sg
LOCQUAL2ERR 0.1 0.5 1.0 2.0 99999.9
LOCGRID 221 291 189 -55.0 -72.5 -1.0 0.5 0.5 0.5 PROB_DENSITY SAVE #LP
LOCPHSTAT 9999.0 -1 9999.0 1.0 1.0 9999.9 -9999.9 9999.9
LOCANGLES ANGLES_NO 5
LOCMAG ML_HB 1.0 1.110 0.00189


**Now the input file to use NLLoc is ready.**

# Merge all in one runfile

In [166]:
# Define the filename for the full runfile
output_file = f'nlloc_svi_{wave}.in'
input_files = glob.glob('*_final.in')[::-1]

# Remove the wave we are not using
remove_suffix = 'S' if wave == 'P' else 'P'
#remove_suffix2 = 'Sg' if wave == 'P' else 'Pg'

# Open the output file for writing
with open(output_file, 'w') as outfile:
    # Iterate over all input files
    for fname in input_files:
        # Open each input file for reading
        with open(fname) as infile:
            lines = infile.readlines()
            for line in lines:
                # Check for the commands LOCPHASEID, VGTYPE, and GTFILES
                if any(cmd in line for cmd in ['VGTYPE', 'GTFILES']): #'LOCPHASEID', 
                    # Skip lines that end with either remove_suffix or remove_suffix2
                    if line.strip().endswith(remove_suffix): #or line.strip().endswith(remove_suffix2):
                        continue  # Skip this line
                # Write the remaining lines to the output file
                outfile.write(line)
            # Add a newline to separate file contents (optional)
            outfile.write('\n')

print(f"All input files merged into {output_file}")

All input files merged into nlloc_svi_P.in


# Final runfile (amazing)

In [167]:
# By alphabetical order to verify the command more easily
with open(output_file, 'r') as file:
    lines = file.readlines()
    non_comment_lines = [line.strip() for line in lines if not line.strip().startswith('#') and line.strip()]
    non_comment_lines_sorted = sorted(non_comment_lines)
    for line in non_comment_lines_sorted:
        print(line)

CONTROL 1 54321
GTFILES  ./model/layer  ./time/layer P
GTMODE GRID3D ANGLES_NO
GT_PLFD  1.0e-3  0
INCLUDE /projects/amt/lpapin/SVI/NLLoc/obs/stations_coords_svi.txt #LP
LOCANGLES ANGLES_NO 5
LOCCOM Bostock_Loc
LOCFILES /projects/amt/lpapin/SVI/NLLoc/obs/LFE_cat_SVI.nlloc NLLOC_OBS /projects/amt/lpapin/SVI/NLLoc/time/layer /projects/amt/lpapin/SVI/NLLoc/loc/SVI_Bostock/family #LP
LOCGAU 0.2 0.0
LOCGAU2 0.01 0.05 2.0
LOCGRID 221 291 189 -55.0 -72.5 -1.0 0.5 0.5 0.5 PROB_DENSITY SAVE #LP
LOCHYPOUT SAVE_NLLOC_ALL SAVE_NLLOC_SUM SAVE_HYPOINV_SUM
LOCMAG ML_HB 1.0 1.110 0.00189
LOCMETH EDT_OT_WT 9999.0 4 -1 -1 -1 -1 -1 1
LOCPHASEID  P   P p Pn Pg
LOCPHASEID  S   S s Sn Sg
LOCPHSTAT 9999.0 -1 9999.0 1.0 1.0 9999.9 -9999.9 9999.9
LOCQUAL2ERR 0.1 0.5 1.0 2.0 99999.9
LOCSEARCH  OCT 8 11 6 0.001 100000 10000 0 0 #Mine
LOCSIG Lois Papin -- UO
TRANS LAMBERT WGS-84 48.65 -123.75 48.0 49.3 0.0 #LP
VGGRID 111 146 95 -55.0 -72.5 -1.0 1.0 1.0 1.0 SLOW_LEN #LP
VGINP /projects/amt/lpapin/SVI/NLLoc/model/Sa

# Notes

*Here's some notes on the location process:* 

What have been used:
- Stations BH* and HH* from networks CN, C8, PO;
- 3D Velocity Model from Savard for Vp and Vs on Southern Vancouver Island: the area has been shrunk to encompass the stations we are using for the locations;
- Picks of Bostock families taken in Tim's NonLinLoc folder: 130 families from number 001 to 300 with P&S waves picks.

What have been done:
- Vel2Grid3d and Grid2Time on runfiles for both P&S waves to create the travel-time grids;
- NLLoc on the P-wave runfile (works the same with S) with the PROB_DENSITY statistical quantity to compute (LOCGRID command), and the Octtree sampling algorithm (LOCSEARCH command).

What we get:
- Velocity grid and travel-time grid for the choosen stations;
- Location files for each Bostock's families in a folder in different formats.

What is next:
- Adapt one of Tim's code to plot the results;
- Modifying the searching algorithm parameters to see how different the locations are;
- Use my picks on Bostock's families with the stacking detections to see if it helps or no;
- Finally use SW4 with the locations (yes!!).